In [ ]:
#Importing necessary packages
import cv2
import os
from tensorflow.keras.models import load_model
import numpy as np

In [ ]:
%cd "C:\Users\praga\OneDrive\Desktop"

In [ ]:
#Loading the trained CNN model.
model_load = load_model("mask_recog_ver2.h5")

In [ ]:
faceCascadeClf = cv2.CascadeClassifier('C:\\Users\\praga\\Anaconda3\\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_alt2.xml')
model = load_model("mask_recog_ver2.h5")
 
video_capture = cv2.VideoCapture(0) #Starting the webcam for video streaming
while True:           #Running infinite loop till the webcam is on to dettect faces and masks on them.
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascadeClf.detectMultiScale(gray,
                                         scaleFactor=1.05,
                                         minNeighbors=6,
                                         minSize=(30, 30),
                                         flags=cv2.CASCADE_SCALE_IMAGE)    
    faces_list=[]
    preds=[]
    for (x, y, w, h) in faces:
        face_frame = frame[y:y+h,x:x+w]  #Co-ordinates of the 
        face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
        face_frame = cv2.resize(face_frame, (300, 200))  #Resizing as the model is trained on images of this size.

        face_frame = np.expand_dims(face_frame, axis=0)  
        face_frame = np.asarray(face_frame)  #Converting the images to array.
        face_frame = face_frame.astype('float32')  
        face_frame /= 255  #Normalising the data.
        faces_list.append(face_frame)
        if len(faces_list)>0:
            preds = model.predict_classes(faces_list) #Predict if there is mask on the image/frame.
        for pred in preds:
            
            (k) = pred
        label = "Mask" if k > 0 else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)  #The color is green for Mask 
        label = "{}".format(label)
        cv2.putText(frame, label, (x, y- 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
 
        cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)
        # Display the resulting frame
    cv2.imshow('Video', frame)
    k = cv2.waitKey(1)  #Will display a frame for 1 ms, after which display will be automatically closed.
    if k == 50: 
        cv2.destroyAllWindows() 
    elif k == ord('k'): # Use "s" key to exit.
        video_capture.release()  #Close the window
        cv2.destroyAllWindows()  